In [1]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION = "-c search_path=dev"

MoM FFBL Weekly Rankings Replacement 

In [2]:
"""
MoM FFBL Weekly Rankings Replacement 2pt system
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390"""
matchups = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(matchups_query)
teams = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(teams_query)
matchups = matchups[
    (matchups["is_playoffs"] == 0)
    & (matchups["is_consolation"] == 0)
    & (matchups["game_id"] >= 390)
]
matchups_a = matchups.copy()
matchups_b = matchups.copy()
matchups_b_cols = list(matchups_b.columns)
rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"
matchups_b.rename(columns=rename_columns, inplace=True)
matchups = pd.concat([matchups_a, matchups_b])
matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)
matchups.reset_index(drop=True, inplace=True)
matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
    ],
    axis=1,
    inplace=True,
)
matchups["win_loss"] = np.where(
    matchups["winner_team_key"] == matchups["team_a_team_key"], "W", "L"
)
matchups["2pt_system"] = matchups.groupby(["week", "game_id"])["team_a_points"].rank(
    "first", ascending=False
)
matchups["2pt_system"] = np.where(matchups["2pt_system"] <= 5, 1, 0)
matchups = matchups[
    [
        "week",
        "team_a_team_key",
        "team_a_points",
        "team_a_projected_points",
        "win_loss",
        "2pt_system",
        "game_id",
    ]
]
weeks = list(matchups["week"].unique())
regular_season = matchups[matchups["week"] == 1]
for wk in weeks[1:]:
    reg_season_week = matchups[matchups["week"] == wk]
    regular_season = regular_season.merge(
        reg_season_week,
        how="outer",
        on="team_a_team_key",
        copy=True,
        suffixes=("", f"_{wk}"),
    )
reg_season_cols = list(regular_season.columns)
twopt_cols = []
team_pts = []
team_pro_pts = []
w_l_cols = []
for col in reg_season_cols:
    if "2pt_system" in col:
        twopt_cols.append(col)
    elif "team_a_points" in col:
        team_pts.append(col)
    elif "team_a_projected_points" in col:
        team_pro_pts.append(col)
    elif "win_loss" in col:
        w_l_cols.append(col)
regular_season.insert(2, "team_a_points_total", regular_season[team_pts].sum(axis=1))
regular_season.insert(
    3, "team_a_project_points_total", regular_season[team_pro_pts].sum(axis=1)
)
regular_season.insert(
    4, "wins", regular_season[w_l_cols].apply(lambda s: (s == "W").sum(), axis=1)
)
regular_season.insert(
    5, "losses", regular_season[w_l_cols].apply(lambda s: (s == "L").sum(), axis=1)
)
twopt_cols.append("wins")
regular_season.insert(2, "2pt_total", regular_season[twopt_cols].sum(axis=1))
week_1 = regular_season["week"]
regular_season.drop("week", axis=1, inplace=True)
regular_season.insert(6, "week_1", week_1)
regular_season.insert(
    2,
    "2pt_rank",
    regular_season.groupby(["game_id"])["2pt_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    4,
    "pts_rank",
    regular_season.groupby(["game_id"])["team_a_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    6,
    "projected_pts_rank",
    regular_season.groupby(["game_id"])["team_a_project_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    0,
    "tuple",
    regular_season[["2pt_rank", "pts_rank"]].apply(tuple, axis=1),
)
regular_season.insert(
    0,
    "overall_rank",
    regular_season.groupby(["game_id"])["tuple"]
    .rank(method="min", ascending=True)
    .astype(int),
)
regular_season = regular_season.merge(
    teams, how="outer", left_on="team_a_team_key", right_on="team_key"
)
team_name = regular_season["name"]
team_mang = regular_season["nickname"]
game_id = regular_season["game_id_y"]
regular_season.drop(
    ["team_a_team_key", "name", "nickname", "tuple", "game_id_x", "game_id_y"],
    axis=1,
    inplace=True,
)
regular_season.insert(1, "manager", team_mang)
regular_season.insert(2, "team_name", team_name)
regular_season.insert(0, "game_id", game_id)
regular_season.sort_values(
    ["overall_rank", "2pt_total", "team_a_points_total"],
    ascending=[True, False, False],
    inplace=True,
)
reg_season_2pt = regular_season[['game_id', 'overall_rank', 'manager', 'team_name', '2pt_total', '2pt_rank', 'team_a_points_total', 'pts_rank', 'team_a_project_points_total', 'projected_pts_rank', 'wins', 'losses']]
DatabaseCursor.copy_table_to_postgres_new(reg_season_2pt, "regseasonrankings2pt", first_time='yes')

Successfully pulled: SELECT * FROM dev.weeklyleaguematchups
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390


#MoM FFBL Draft Analysis/Trends Replacement

In [3]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT team_key
, game_id
, league_id
, "team_standings.outcome_totals.wins"
, "team_standings.outcome_totals.losses"
, number_of_moves
, number_of_trades
, nickname
, name
, draft_grade
, clinched_playoffs 
, "team_standings.playoff_seed"
, "team_standings.points_against" 
, "team_standings.points_for"
, "team_standings.rank"
FROM dev.leagueteams"""
players_query = """SELECT player_key
, "name.full"
, league_id
, game_id
, primary_position
, "bye_weeks.week"
, "draft_analysis.average_pick"
, "draft_analysis.average_round"
, "draft_analysis.percent_drafted"
 FROM dev.playerlist"""
draft = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(teams_query)
players = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(players_query)
draft_analysis = draft.merge(teams, how='left', left_on=["team_key", "game_id", "league_id"], right_on=["team_key", "game_id", "league_id"], suffixes=("_draft", "_teams"))
draft_analysis = draft_analysis.merge(players, how='left', left_on=["player_key", "game_id", "league_id"], right_on=["player_key", "game_id", "league_id"], suffixes=("", "_players"))
draft_analysis = draft_analysis[~draft_analysis['name'].isna()]
draft_analysis['clinched_playoffs'].fillna(0, inplace=True)
draft_analysis['team_standings.playoff_seed'].fillna(0, inplace=True)
encoded_grades = {
    "A+": 4,
    "A": 4,
    "A-": 3.7,
    "B+": 3.3,
    "B": 3.0,
    "B-": 2.7,
    "C+": 2.3,
    "C": 2.0,
    "C-": 1.7,
    "D+": 1.3,
    "D": 1.0,
    "na": np.nan,
}
draft_analysis["draft_gpa"] = draft_analysis["draft_grade"].replace(encoded_grades)
draft_analysis["avg_draft_gpa_rank"] = draft_analysis.groupby(["team_standings.rank"])["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_nickname"] = draft_analysis.groupby(["nickname"])["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_clinched_playoffs"] = draft_analysis.groupby(["clinched_playoffs"])["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_playoffseed"] = draft_analysis.groupby(["team_standings.playoff_seed"])["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_draft_pos"] = draft_analysis.groupby(["pick"])["draft_gpa"].transform("mean")

Successfully pulled: SELECT * FROM dev.draftresults
Successfully pulled: SELECT team_key
, game_id
, league_id
, "team_standings.outcome_totals.wins"
, "team_standings.outcome_totals.losses"
, number_of_moves
, number_of_trades
, nickname
, name
, draft_grade
, clinched_playoffs 
, "team_standings.playoff_seed"
, "team_standings.points_against" 
, "team_standings.points_for"
, "team_standings.rank"
FROM dev.leagueteams
Successfully pulled: SELECT player_key
, "name.full"
, league_id
, game_id
, primary_position
, "bye_weeks.week"
, "draft_analysis.average_pick"
, "draft_analysis.average_round"
, "draft_analysis.percent_drafted"
 FROM dev.playerlist


MoM FFBL League History Replacement

In [22]:
"""
MoM FFBL League History Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT * FROM dev.leagueteams"""
draft_query = """SELECT * FROM dev.draftresults"""
players_query = """SELECT * FROM dev.playerlist"""
matchups = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(matchups_query)
teams = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(teams_query)
draft = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(draft_query)
players = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(players_query)
matchups = matchups.merge(teams, how='left', left_on=['game_id', 'league_id', 'team_a_team_key'], right_on=['game_id', 'league_id', 'team_key'], suffixes=('', '_team_a'))
matchups = matchups.merge(teams, how='left', left_on=['game_id', 'league_id', 'team_b_team_key'], right_on=['game_id', 'league_id', 'team_key'], suffixes=('_team_a', '_team_b'))

Successfully pulled: SELECT * FROM dev.weeklyleaguematchups
Successfully pulled: SELECT * FROM dev.leagueteams
Successfully pulled: SELECT * FROM dev.draftresults
Successfully pulled: SELECT * FROM dev.playerlist


In [32]:
matchups[(matchups["is_playoffs"] == 1) & (matchups["is_consolation"] == 0)].sort_values(['week_start', 'is_consolation', 'winner_team_key'], ascending=[False, True, True])

,is_consolation,is_matchup_recap_available,is_playoffs,is_tied,matchup_recap_title,matchup_recap_url,status,week,week_end,week_start,winner_team_key,team_a_grade,team_a_team_key,team_a_points,team_a_projected_points,team_b_grade,team_b_team_key,team_b_points,team_b_projected_points,game_id,league_id,clinched_playoffs_team_a,draft_grade_team_a,faab_balance_team_a,has_draft_grade_team_a,manager_id_team_a,name_team_a,nickname_team_a,number_of_moves_team_a,number_of_trades_team_a,roster_adds.coverage_value_team_a,roster_adds.value_team_a,team_id_team_a,team_key_team_a,team_standings.outcome_totals.losses_team_a,team_standings.outcome_totals.percentage_team_a,team_standings.outcome_totals.ties_team_a,team_standings.outcome_totals.wins_team_a,team_standings.playoff_seed_team_a,team_standings.points_against_team_a,team_standings.points_for_team_a,team_standings.rank_team_a,team_standings.streak.type_team_a,team_standings.streak.value_team_a,waiver_priority_team_a,clinched_playoffs_team_b,draft_grade_team_b,faab_balance_team_b,has_draft_grade_team_b,manager_id_team_b,name_team_b,nickname_team_b,number_of_moves_team_b,number_of_trades_team_b,roster_adds.coverage_value_team_b,roster_adds.value_team_b,team_id_team_b,team_key_team_b,team_standings.outcome_totals.losses_team_b,team_standings.outcome_totals.percentage_team_b,team_standings.outcome_totals.ties_team_b,team_standings.outcome_totals.wins_team_b,team_standings.playoff_seed_team_b,team_standings.points_against_team_b,team_standings.points_for_team_b,team_standings.rank_team_b,team_standings.streak.type_team_b,team_standings.streak.value_team_b,waiver_priority_team_b
79,0,1,1,0,Back-and-forth battle ends with Dak ...,https://football.fantasysports.yahoo...,postevent,17,2022-01-03,2021-12-28,406.l.103661.t.2,C,406.l.103661.t.2,103.94,124.62,C,406.l.103661.t.4,101.16,129.30,406,103661,1.0,B-,0.0,1.0,2.0,Dak to the Future,Wes,30.0,2.0,18.0,0.0,2.0,406.l.103661.t.2,7.0,0.53,0.0,8.0,4.0,1630.12,1648.42,3.0,loss,2.0,1.0,1.0,A-,0.0,1.0,4.0,King of the Hill,Ryan,54.0,0.0,18.0,0.0,4.0,406.l.103661.t.4,5.0,0.67,0.0,10.0,2.0,1600.38,1854.08,4.0,win,4.0,2.0
78,0,1,1,0,Harris & Harris RBs at Law 👨🏾‍⚖️ def...,https://football.fantasysports.yahoo...,postevent,17,2022-01-03,2021-12-28,406.l.103661.t.7,B+,406.l.103661.t.1,130.70,125.13,A+,406.l.103661.t.7,162.82,131.13,406,103661,1.0,B,0.0,1.0,1.0,Fantasy Sidelines,Tim,73.0,2.0,18.0,0.0,1.0,406.l.103661.t.1,4.0,0.73,0.0,11.0,1.0,1658.44,1725.32,2.0,win,1.0,8.0,1.0,C,34.0,1.0,7.0,Harris & Harris RBs at Law 👨🏾‍⚖️,Greg,53.0,0.0,18.0,0.0,7.0,406.l.103661.t.7,6.0,0.60,0.0,9.0,3.0,1560.30,1799.50,1.0,loss,1.0,7.0
74,0,1,1,0,Fantasy Sidelines defeat Dak to the ...,https://football.fantasysports.yahoo...,postevent,16,2021-12-27,2021-12-22,406.l.103661.t.1,B,406.l.103661.t.1,114.98,103.87,D,406.l.103661.t.2,88.50,128.69,406,103661,1.0,B,0.0,1.0,1.0,Fantasy Sidelines,Tim,73.0,2.0,18.0,0.0,1.0,406.l.103661.t.1,4.0,0.73,0.0,11.0,1.0,1658.44,1725.32,2.0,win,1.0,8.0,1.0,B-,0.0,1.0,2.0,Dak to the Future,Wes,30.0,2.0,18.0,0.0,2.0,406.l.103661.t.2,7.0,0.53,0.0,8.0,4.0,1630.12,1648.42,3.0,loss,2.0,1.0
76,0,1,1,0,King of the Hill fall to Harris & Ha...,https://football.fantasysports.yahoo...,postevent,16,2021-12-27,2021-12-22,406.l.103661.t.7,C+,406.l.103661.t.4,112.92,132.40,B,406.l.103661.t.7,125.00,123.71,406,103661,1.0,A-,0.0,1.0,4.0,King of the Hill,Ryan,54.0,0.0,18.0,0.0,4.0,406.l.103661.t.4,5.0,0.67,0.0,10.0,2.0,1600.38,1854.08,4.0,win,4.0,2.0,1.0,C,34.0,1.0,7.0,Harris & Harris RBs at Law 👨🏾‍⚖️,Greg,53.0,0.0,18.0,0.0,7.0,406.l.103661.t.7,6.0,0.60,0.0,9.0,3.0,1560.30,1799.50,1.0,loss,1.0,7.0
166,0,1,1,0,"Beats by DeAndre beat expectations, ...",https://football.fantasysports.yahoo...,postevent,16,2020-12-28,2020-12-22,399.l.12514.t.4,A-,399.l.12514.t.4,145.96,125.37,D+,399.l.12514.t.6,95.88,126.11,399,12514,1.0,D+,0.0,1.0,4.0,Beats by DeAndre,Wes,35.0,1.0,17.0,0.0,4.0,399.l.12514.t.4,6.0,0.54,0.0,7.0,5.0,1395.30,1453.86,3.0,loss,1.0,4.0,1.0,B-,32.0,1.0,6.0,Poo

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""